# 🦊 Qdrant quick start search

Author: J Visbeen  
Target: practice  
Source: https://qdrant.tech/documentation/quick-start/

Docker run command voor de container:
```
sudo docker-compose up
```
aanntekeningen maken over ervaringen

In [1]:
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams, PointStruct, Filter, FieldCondition, Range
from sentence_transformers import SentenceTransformer

client = QdrantClient('localhost', port=6333)
encoder = SentenceTransformer('all-MiniLM-L6-v2')


ModuleNotFoundError: No module named 'qdrant_client'

In [31]:
client.recreate_collection(
    collection_name='amazon_products',
    vectors_config=VectorParams(
        size=encoder.get_sentence_embedding_dimension(), 
        distance=Distance.COSINE
        )
)

True

In [32]:
import pandas as pd

df_categories = (
    pd.read_csv('../amazon_categories.csv')
    .dropna()
    .reset_index()
)

df_products = (
    pd.read_csv('../amazon_products.csv')
    .dropna()
    .sample(5000, random_state=42)
    .reset_index()
)

merged_df = pd.merge(df_products, df_categories, left_on='category_id', right_on='id', how='left')
merged_df.drop('category_id', axis=1)
merged_df

,index_x,asin,title,imgUrl,productURL,stars,reviews,price,listPrice,category_id,isBestSeller,boughtInLastMonth,index_y,id,category_name
0,1208301,B0BKZNNL1V,"LOVEVOOK Diaper Bag Backpack, Diaper Backpack ...",https://m.media-amazon.com/images/I/718pt0oLES...,https://www.amazon.com/dp/B0BKZNNL1V,4.6,0,36.99,0.00,36,False,0,35,36,Baby Travel Gear
1,1089181,B0968QPTK8,Seloom 148 PCS Reflective Mailbox Numbers and ...,https://m.media-amazon.com/images/I/71Sr2+igJf...,https://www.amazon.com/dp/B0968QPTK8,4.5,286,14.89,19.99,211,False,50,196,211,Hardware
2,466837,B00NIAULVC,"Spry Xylitol Toothpaste 5oz, Fluoride Toothpas...",https://m.media-amazon.com/images/I/51I687iehe...,https://www.amazon.com/dp/B00NIAULVC,4.7,0,8.49,8.99,126,False,900,112,126,Oral Care Products
3,39647,B0794ZFTDB,2 Pieces Rhinestone Bow Ties Banquet Bowties M...,https://m.media-amazon.com/images/I/81G1Dc1uZu...,https://www.amazon.com/dp/B0794ZFTDB,4.4,0,12.99,13.99,112,False,50,101,112,Men's Accessories
4,485619,B075PTHVKN,Mielle Organics Pomegranate & Honey Leave-In C...,https://m.media-amazon.com/images/I/71RNqb6TfJ...,https://www.amazon.com/dp/B075PTHVKN,4.7,0,12.52,14.99,47,False,10000,45,47,Hair Care Products
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,1419223,B0C2PKGCRG,"Tote Handbag Crossbody Shoulder Bag, Canvas Ny...",https://m.media-amazon.com/images/I/41LJWAda2E...,https://www.amazon.com/dp/B0C2PKGCRG,3.9,0,18.99,0.00,118,False,0,105,118,Women's Handbags
4996,278230,B09WJFJX1W,Simpson Strong Tie APRTA2 Outdoor Accents® ZMA...,https://m.media-amazon.com/images/I/71edOXtsyM...,https://www.amazon.com/dp/B09WJFJX1W,0.0,0,40.39,0.00,141,False,0,127,141,Industrial Hardware
4997,1374809,B0CGLXS6QF,BRL136 Replacement Heads for Philips Beauty La...,https://m.media-amazon.com/images/I/71+-ZFLJnl...,https://www.amazon.com/dp/B0CGLXS6QF,0.0,0,17.99,0.00,53,False,0,51,53,Shaving & Hair Removal Products
4998,318330,B00F3F09NU,Arkon Mounts TAB188L22 Car or Truck Seat Rail ...,https://m.media-amazon.com/images/I/517pYYiwuW...,https://www.amazon.com/dp/B00F3F09NU,4.3,0,32.32,37.95,76,False,0,70,76,Accessories & Supplies


In [33]:
client.upload_points(
    collection_name='amazon_products',
    points=[
        PointStruct(
            id=idx, 
            vector=encoder.encode(product.title + ' ' + product.category_name).tolist(),
            payload=product.to_dict()
        )
        for idx, product in merged_df.iterrows()
    ]
)

In [43]:
hits = client.search(
    collection_name="amazon_products",
    query_vector=encoder.encode("games").tolist(),
    query_filter=Filter(
        must=[
            FieldCondition(key='stars', range=Range(gte=5),)
        ]
    ),
    limit=3,
)
for hit in hits:
    print(hit.payload, "score:", hit.score)

{'asin': 'B0C9R3XWQX', 'boughtInLastMonth': 100, 'category_id': 223, 'category_name': 'Games & Accessories', 'id': 223, 'imgUrl': 'https://m.media-amazon.com/images/I/81P0-xljEML._AC_UL320_.jpg', 'index_x': 859914, 'index_y': 208, 'isBestSeller': False, 'listPrice': 0.0, 'price': 34.99, 'productURL': 'https://www.amazon.com/dp/B0C9R3XWQX', 'reviews': 0, 'stars': 5.0, 'title': 'Cats vs Pickles Mayhem Board Game | Card Game for Kids Families and Adults | Beans Card Game | Exclusive CvP Characters | Easy to Learn | Card Game for Family Fun Night | 2 Players to 8 Players'} score: 0.49795586
{'asin': 'B0BRYN6SPN', 'boughtInLastMonth': 0, 'category_id': 270, 'category_name': 'Toys & Games', 'id': 270, 'imgUrl': 'https://m.media-amazon.com/images/I/71lIzsAYBPL._AC_UL320_.jpg', 'index_x': 561152, 'index_y': 247, 'isBestSeller': False, 'listPrice': 0.0, 'price': 29.99, 'productURL': 'https://www.amazon.com/dp/B0BRYN6SPN', 'reviews': 0, 'stars': 5.0, 'title': 'Capstone Games Aleph Null - Capston